### Setup

In [13]:
%run -i '../scripts/tf_test.py'

In [2]:
import tensorflow as tf
tf.enable_eager_execution()

from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.__version__

'1.13.1'

In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14784590721083732387
]


## Start program

### Select a model

In [14]:
'''
Model 
'''

img_size = [224,224]

mobile_net = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False)
mobile_net.trainable=False

model = tf.keras.Sequential([
    mobile_net,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [29]:
'''
Model 
'''

img_size = [224,224]

app = tf.keras.applications.ResNet50(input_shape=(224, 224, 3))
app.trainable=False

model = tf.keras.Sequential([
  app,
#   tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(2, activation = 'sigmoid')
])

102858752/102853048 [==============================] - 11s 0us/step


In [42]:
'''
Model 
'''

img_size = [320,320]

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,kernel_size=(3,3), strides=(1, 1),activation='relu',input_shape=(320,320,3)),
#     tf.keras.layers.Dense(batch_size, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.sigmoid)
])

In [21]:
'''
Model 
'''

img_size = [320,320]

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(320,320,3)),
    tf.keras.layers.Dense(2, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.sigmoid)
])

### Load Data

In [15]:
'''
Set Params

sample_size: 0-1
'''

sample_size = 1.0

batch_size = 32

In [16]:
'''
Load CSVs
'''

df = GetCSVs(sample_size)
print(df['Catenary'].value_counts())

img_paths,labels = GetPaths(df)

[Errno 2] File b'../data/output_images/China/CHN.csv' does not exist: b'../data/output_images/China/CHN.csv'
1    466
0    466
Name: Catenary, dtype: int64
932


In [18]:
'''
Split, shuffle, and batch images into train/test datasets
'''

train_image_label_ds, test_image_label_ds = SplitDataSet(img_paths, labels)

train_ds = ShuffleBatch(train_image_label_ds,len(img_paths),BATCH_SIZE = batch_size) 
test_ds = ShuffleBatch(test_image_label_ds,len(img_paths),BATCH_SIZE = batch_size)

(224, 224, 3)
(224, 224, 3)


### Compile, Fit, and Evaluate

In [19]:
model.compile(optimizer='rmsprop',
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [20]:
steps_per_epoch = int(tf.ceil(len(img_paths)/batch_size).numpy())

model.fit(train_ds,steps_per_epoch=steps_per_epoch, epochs=10)

model.evaluate(test_ds,steps=steps_per_epoch)

Epoch 1/10
Instructions for updating:
Use tf.cast instead.
30/30 [==============================] - 74s 2s/step - loss: 0.7416 - acc: 0.4990
Epoch 2/10
30/30 [==============================] - 60s 2s/step - loss: 0.6974 - acc: 0.5281
Epoch 3/10
30/30 [==============================] - 52s 2s/step - loss: 0.6822 - acc: 0.5583
Epoch 4/10
30/30 [==============================] - 52s 2s/step - loss: 0.6541 - acc: 0.5906
Epoch 5/10
30/30 [==============================] - 52s 2s/step - loss: 0.6361 - acc: 0.6073
Epoch 6/10
30/30 [==============================] - 52s 2s/step - loss: 0.6279 - acc: 0.6542
Epoch 7/10
30/30 [==============================] - 52s 2s/step - loss: 0.6061 - acc: 0.6271
Epoch 8/10
30/30 [==============================] - 54s 2s/step - loss: 0.5846 - acc: 0.6604
Epoch 9/10
30/30 [==============================] - 56s 2s/step - loss: 0.5913 - acc: 0.6562
Epoch 10/10
30/30 [==============================] - 65s 2s/step - loss: 0.7175 - acc: 0.5688


[0.7175362467765808, 0.56875]